# Object-Orientation and Design Patterns

This is the preliminary content for the next talk, I'll refactor this stuff into slides later.


 
Subjects to cover:

 * __Object Definition__: the structural concept of an entity with data and operations
   * Creation
   * Referencing objects and identity
   * Everything is an object
   
 * __Classes__: definition of the object blueprint
 
 * __Class Members (or features)__:
   * Attributes: data components of a class
   * Constructors: how objects are constructed
   * Methods: definition of object operations
   
 * __Notion of Interface__: the layer through which clients interact with objects
   * Abstraction
   * Programming to interfaces
   
 * __Inheritance__: definition of classes in terms of others
   * Concept of specialization
   * Single inheritance
   * Multiple inheritance
   * When to use composition vs. inheritance
   * Principle of Substitution (and duck typing)
   * Inheritance of convenience: don't do this
   
 * __Method Overriding__: modifying behaviour of inherited code by replacing methods
 * __Things Not Present in Python__: method/constructor overloading, variable polymorphism
 
 * Design Patterns:
   * __Subject-Observer__
   * __Iterator__
   * __Template Method__
   * __Factory Method__
   * __Adapter__
 
Key concepts:
 * __Encapsulation__: definitions of data entities are packaged together with the code which actuates their behaviour
 * __Abstraction__: the details of how objects function are hidden from clients while still permitting use
 * __Inheritance__: object definitions can be stated as extensions of existing ones, assuming their members for reuse in the new type
 * __Genericity__: code and data structures can operate with existing types or new ones introduced in later client code
 * __Polymorphism__: objects have multiple types, can have differing behaviour from what would be expected of one of these types 
 * __Substitutability__: objects of a subtype should be substitutable with objects of the parent type while preserving correctness
 

Object-orientation is a programming paradigm centered on the concept of encapsulating data with operations. In the simplest form, an object is a piece of data with associated routines which manipulate that data. Object-oriented systems are composed of many objects aggregating together to form structures and cooperate in implementing a program's behaviour. 

This contrasts with imperative languages like Fortran or C which are composed of routines and data defined separately as global or local variables. Here there is a loose relationship between data and the code that manipulates it, and the structure of the code is much less flexible or reusable.

Data structures and their associated code are defined as abstract data types (ADTs) which are composed of a definition for a data entity and routines which manipulate such entities. For example, a `Dimension` type in C to represent a 2D area:

```
struct Dimension { int width,height; }
void init(struct Dimension* d, int w, int h);
int area(struct Dimension* d);
```

The abstract component of this ADT is evident in that the data entity and its routines are declared without being befined. 
A client can still use this ADT without knowing the details of its implementation; abstraction is an important property to ensure a separation of concerns and modularity in code.
This however defines a weak link between the data entity `Dimension` and the routines, as well as creating a definition which cannot be changed or adapted in client code.

Object-oriented programming aims to make the connection between data and code tighter and explicit while preserving abstraction. Objects are instances of ADTs which encapsulate data and routines in one entity, as demonstrated in the C++ equivalent of the above:

```
class Dimension {
private:
  int width,height;
public:
  Dimension(int w, int h);
  virtual int area();
};
```

In Python:

In [34]:
class Dimension(object):
    def __init__(self,w,h):
        self.width=w
        self.height=h
    def area(self):
        return self.width*self.height

An object is created by instantiating the class, that is creating an instance of class which has those members it defines:

In [35]:
d=Dimension(10,20)

The variable `d` references an instance of `Dimension`, therefore this object has the type `Dimension` as well as `object`. Multiple instances of a class can be created, each instance is an independent object with their own unique identities and distinct members:

In [36]:
d1=Dimension(15,30)
print id(d),id(d1)

61323416 61323024


In Python it is important to note that assigning to a variable only causes it to refer to a new object, that is to say a variable is a name that is bound to an object. This doesn't copy the object in any way but may make the previous referenced object inaccessible:

In [37]:
d1=d
print id(d),id(d1)

61323416 61323416


Each object has the members defined by the class. Members (or features in some literature) are the components of an object falling into these broad categories:

 * __Attributes__: named data values stored in the object
 * __Methods__: routines associated with the object and which can refer to the object by name
 * __Constructor__: special method used to setup a new object's state at the point of instantiation
 
Other types of members exist in other languages but they are special forms of either attributes or methods. A member of an object is accessed using the dot notation of the form __`object.member`__. This allows members to be accessed and mutated:

In [38]:
print d.width, d.height
d.width=12
print d.width, d.height
print d.area # methods can be accessed without being called

10 20
12 20
<bound method Dimension.area of <__main__.Dimension object at 0x0000000003A7B898>>


One important name that exists in all methods is __self__ which refers to the object whose method was called. Recall the definition of `area()` from `Dimension`:

In [39]:
def area(self):
    return self.width*self.height

When a method of an object is called using the dot notation, the value for `self` is set to the object (called the receiver or callee) within the scope of the call. Thus when calling `area()` with `d` as the receiver, `self` is bound to `d` and so allows access to its members:

In [40]:
print d.area()

240


Calling this method is equivalent to calling a function and passing `d` as the first argument whose members are then accessed in the calculation

In [43]:
def areafunc(self):
    return self.width*self.height

print areafunc(d)

240


This however breaks the relationship between instances of `Dimension` and the operation of calculating an area, especially in Python which does not assign a type to argument values. Nothing prevents the following, which is obviously wrong but only discoverable at runtime:

In [44]:
print areafunc('I am not a Dimension object')

AttributeError: 'str' object has no attribute 'width'

A method defines an operation which is associated with an object, but it also defines an interface in terms of the process of calling it. The `area()` method for example defines an interface as a method named `area` which accepts no arguments and returns a single number. This represents all the information a caller (or client) needs to know about `area()` to be able to use it, that is the interface abstracts away the details of implementation. The interfaces of all the methods for an object, as well as the notions of accessing and mutating attributes, aggregate together to form the __object interface__ for that object. 

A client, whether another object or a routine, need only know about an object's interface to be able to interact with it. Since the interface is abstract there are no requirements regarding the type or implementation of this object, thus two objects can have the same interface but differing implementations. For example, any other class which defines an `area()` method with the same arguments and return type essentially defines the same interface:

In [46]:
class Dimension3(object):
    def __init__(self,w,h,d):
        self.width=w
        self.height=h
        self.depth=d
        
    def area(self):
        return 2*(self.width*(self.height+self.depth)+self.height*self.depth)
    
d3=Dimension3(10,12,15)
print d3.area()

900


If an instance of `Dimension` and another of `Dimension3` both implement the same interface (considering `area()` only), then a client can be implemented which function with either:

In [47]:
def calcsquare(obj):
    area=obj.area()
    return int(area**0.5)

print calcsquare(d)
print calcsquare(d3)

15
30


The function `calcsquare()` uses only the interface of `object` to interact with it, which is either `Dimension.area` or `Dimension3.area` depending on what object was passed in as `obj`. Defining clients in terms of interfaces is called __Programming to Interfaces__ naturally enough. It is an important mechanism for reuse since it allows routines, algorithms, or whole modules to be integrated with introduced code whose implementations are unknown (for example your code). 

In Python this can be done easily since `calcsquare()` doesn't check that its argument `obj` fulfills the interface it needs, it just tries to call `area()` and if it isn't suitable an error occurs at runtime. This is called __duck typing__ since if it looks like a duck, and quacks like a duck, it ain't a moose. In our example having the correct `area()` method constitutes quacking in the right manner.

In other languages like C++, C#, or Java which are statically typed there must be a type which defines the needed interface, and any object must inherit this type to be usable. For example in C++:

```
class AreaInterface { virtual float area()=0; };
float calcsquare(const AreaInterface& a) { ... }
```

__Inheritance__ allows a class to be defined in terms of another, the specific rules for which vary by language but in general the inheriting type (a subtype or subclass) receives all the members of the type being inherited (the supertype or superclass). This allows a class to acquire code members without having to redefine them, so prevents reinventing the wheel in many cases and so is an important component to reuse. Consider a class defining a specific rectangular area in 2D space:

In [51]:
class Rect(Dimension):
    def __init__(self,x,y,w,d):
        Dimension.__init__(self,w,d)
        self.x=x
        self.y=y
        
    def farCorner(self):
        return (self.x+self.width,self.y+self.height)
    
r=Rect(4,4,12,10)
print r.x, r.y, r.width, r.height
print r.area(), r.farCorner()
print isinstance(r,Rect), isinstance(r,Dimension)

4 4 12 10
120 (16, 14)
True True


The class `Rect` has inherited members from `Dimension` and introduced new ones in its constructor. It has an `area()` method which functions as before as well as defined a new method. What is important to note is that instances of `Rect` are also instances of `Dimension`; this is one aspect of __polymorphism__. 

Inheritance is more than just the copy-pasting of members, or at least should be. The idea with inheritance is that the subtype is a __specialization__ of the supertype, it represents a related concept that is more refined or specific to a particular context. In our example, a rectangle has a notion of dimension as well as location and so is a special type of dimension. Other subtypes of `Dimension` could be defined which represent some other specialization but which has no relation to `Rect`. Classes can also inherit from multiple supertypes, thus should be thought of as being specialization of multiple concepts at once.